### Title: Automated Forex News Scalping Trading Bot
Author: Tan Zhi Lun  
Contact: zhilun296@gmail.com

## The Strategy

Imagine that we are presented with the oldest bet in the book, a coinflip. When the coin is in the air, we have no way of knowing whether the coin will land on heads, or tails.  
But what if we knew the exact point in **time** that the coin will land?

Well, then we could say, with a 100% certainty, that at that exact point in time the coin's state will be either heads or tails. And we see that this is what is happening in the forex market, as countries' economic news are always released at a pre-specified time. For example, non-farm payrolls, which is one of the most closely monitored forex news is always released at 2030 hrs (SGT) on the first Friday of every month. These news events often bring volatility, and more importantly, it may cause a strong candle to form due to mass buying/ selling if the result is extremely unexpected.  
As shown, the news are always released at a predictable time:

<img src='./Images/forexnews.jpg' width="600">

Note that what matters here is that the result must be significantly different from the **consensus**; it does not matter in terms of the value of the result itself. For example, assume that the US economy has been doing relatively poorly and the estimates for the NFP is at 1360K, down from 4800K last month as shown above. If the actual results show 2000K, the market would still be **bullish** on USD even though it is down from 4800K, whereas an upset to the downside at 1000K would potentially cause a huge **bearish** candle, off of which we could capitalize on.  

So, the strategy here is:
1. Place both a buy stop and a sell stop at a certain number of pips above and below the current price two minutes prior to news strike (S/L: current price).
2. If the result is not significantly different from the consensus, close off both positions.
3. If the buy stop is triggered, close the sell stop position and trail the stop loss. Vice versa if sell stop is triggered.

Below is an illustration of the strategy (news time is at 9pm):

<img src='./Images/stratpic1.jpg' width="600">

<img src='./Images/stratpic2.jpg' width="600">

<img src='./Images/stratpic3.jpg' width="600">

## Discussion
The discussion will be centered around 2 main points:
1. Implementation of the strategy
2. Optimization of strategy parameters

## 1. Implementation of the strategy
The strategy is implemented using Darwinex's API, which is built on the ZeroMQ Connector to MT4. The reason why this API is chosen is mainly because of flexibility; as this API allows us to choose from any broker that uses MT4 as it is built upon MQL4. Thus, we would be able to choose for example, Pepperstone as our broker as their flat commission rate structure instead of a bid-ask spread would allow us to avoid the potentially higher bid-ask spread during the news strike time.  

In a nutshell, the API works by connecting external scripts in other languages such as Python to MT4 through synchronous and asynchronous messaging using Expert Advisors MQL4 script. For further details, please refer to Darwinex's github: https://github.com/darwinex/dwx-zeromq-connector

The following code is built upon this file in the API: https://github.com/darwinex/dwx-zeromq-connector/blob/master/v2.0.1/python/examples/template/strategies/base/DWX_ZMQ_Strategy.py .

```python
# Presented in markdown format as the files required would be too heavy for the repo.
# For further details please refer to the github of the API.

# The _trader_ method is written in the Strategy class.
# Symbol is a list, where index 0 is currency pair, index 1 is lot size.

    def _trader_(self, _symbol, _max_trades):     
        # Construct first order form, which will be sent to MT4 for execution later on. Order type 4 is buy stop.
        # Price will be set later on
        _default_order_1 = self._zmq._generate_default_order_dict()
        _default_order_1['_type'] = 4
        _default_order_1['_symbol'] = _symbol[0]
        _default_order_1['_lots'] = _symbol[1]
        _default_order_1['_SL'] = _default_order_1['_TP'] = 120
        _default_order_1['_comment'] = '{}_Trader'.format(_symbol[0])
        _default_order_1['_magic'] = 1
        
        # Construct second order form, which will be sent to MT4 for execution later on. Order type 5 is sell stop.
        # Price will be set later on.
        _default_order_2 = self._zmq._generate_default_order_dict()
        _default_order_2['_type'] = 5
        _default_order_2['_symbol'] = _symbol[0]
        _default_order_2['_lots'] = _symbol[1]
        _default_order_2['_SL'] = _default_order_2['_TP'] = 120
        _default_order_2['_comment'] = '{}_Trader'.format(_symbol[0])
        _default_order_2['_magic'] = 2
        
        # Subscribe to market data for execution
        self._zmq._DWX_MTX_SUBSCRIBE_MARKETDATA_(_symbol=_symbol[0])
        
        # Define variables to be updated later in the loop
        newstimepricehigh = None
        newstimepricelow = None
        buystopticket = None
        sellstopticket = None
        value_sell = None
        value_buy = None
        time_diff = None
        pricelist = []
        
        # Main bulk of the strategy
        while self._market_open:
   
                # Variables regarding time
                newstime = datetime(2020,8,3,8,42)
                newstimeminus2 = newstime - Timedelta(minutes=2)
                timedifftoorder = datetime.now() - newstimeminus2
                currenttime = datetime.now()
                time_buffer = 30
                fiveminsbuffer = newstime + Timedelta(minutes=5)
            
                # This gives us a order matrix, and this is used to calculate number of open trades
                _ot = self._reporting._get_open_trades_('{}_Trader'.format(_symbol[0]),
                                                            self._delay,
                                                            10)
                
                # If there are no open trades (buy stop and sell stop are also considered open trades)
                # And it is two minutes before news time
                # Note that this section is entirely before news time
                # Central idea is to put in the buy and sell stop.
                if _ot.shape[0] == 0 and 0 < timedifftoorder.total_seconds() < time_buffer:
                    
                    try:                        
                        # Get current price
                        Temp_CurrBidAsk = self._zmq._Market_Data_DB[_symbol[0]].items()
                        CurrBidAsk = list(Temp_CurrBidAsk)[-1][1]
                        
                        # These are the buy stop and sell stop entry prices, currently set at 12 pips above and below.
                        newstimepricehigh = (CurrBidAsk[0] + CurrBidAsk[1])/2 + 0.0012
                        newstimepricelow = newstimepricehigh - 0.0024
                        
                        # Set price for order form 1
                        _default_order_1['_price'] = newstimepricehigh
                        # Set buy stop using order form 1
                        _ret = self._execution._execute_(_default_order_1,
                                                         self._verbose,
                                                         self._delay,
                                                         10) 
                        # Set price for order form 2
                        _default_order_2['_price'] = newstimepricelow                       
                        # Set sell stop using order form 2
                        _retto = self._execution._execute_(_default_order_2,
                                                         self._verbose,
                                                         self._delay,
                                                         10)
                        # Get ticket for this buy stop such that we can interact with it later on.
                        buystopticket = _ret['_ticket'] 
                        # Get ticket for this buy stop such that we can interact with it later on. 
                        sellstopticket = _retto['_ticket']
                        # Get news time price which will be used for the trailing stop loss
                        newstimeprice=(newstimepricehigh+newstimepricelow)/2
                        pricelist.append(newstimeprice)                        
                        continue           
                    except:     
                        continue     
               
                # After news time strike
                # Note that this section is entirely after news time
                # The central idea is to close unwanted trades and trail stop loss.
                elif currenttime > newstime:
                    # Variables to calculate current price
                    Temp_CurrBidAsk2 = self._zmq._Market_Data_DB[_symbol[0]].items()                    
                    CurrBidAsk2 = list(Temp_CurrBidAsk2)[-1][1]
                    currentprice = (CurrBidAsk2[0] + CurrBidAsk2[1])/2
                    pricelist.append(currentprice)
                    time_diff = (Timedelta(currenttime - newstime).total_seconds())//120
                        
                    # If sell stop is triggered, trail stop loss by moving it down
                    # The stop loss slowly shrinks and "converges" to the current price (floored at 6 pips)
                    # Close buy stop by using its ticket
                    if currentprice < newstimepricelow:
                        if len(pricelist)>=2 and pricelist[-1] == min(pricelist):
                            try:
                                value_sell = round(((currentprice - \
                                                     (newstimepricelow-max((0.0012time_diff*0.0002),0.0006)))*100000),2)
                                self._zmq._DWX_MTX_MODIFY_TRADE_BY_TICKET_(sellstopticket,value_sell,10000)
                            except:
                                pass         
                        try:
                            self._zmq._DWX_MTX_CLOSE_TRADE_BY_TICKET_(buystopticket)
                        except:
                            pass                         
                        sleep(1)                        
                        continue
                        
                    # If buy stop is triggered, trail stop loss by moving it up
                    # The stop loss slowly shrinks and "converges" to the current price (floored at 6 pips)
                    # Close sell stop by using its ticket
                    elif currentprice > newstimepricehigh:
                        if len(pricelist)>=2 and pricelist[-1] == max(pricelist):
                            try:
                                value_buy = round(((newstimepricehigh - \
                                                    (currentprice - max((0.0012-time_diff*0.0002),0.0006)))*100000),2)
                                self._zmq._DWX_MTX_MODIFY_TRADE_BY_TICKET_(buystopticket,value_buy,10000)
                            except:
                                pass                            
                            try:
                            self._zmq._DWX_MTX_CLOSE_TRADE_BY_TICKET_(sellstopticket)
                        except:
                            pass
                        sleep(1)
                        continue             
                    
                    # If 5 minutes has passed without anything happening, close all trades.
                    elif currenttime > fiveminsbuffer:
                        self._zmq._DWX_MTX_CLOSE_ALL_TRADES_()
                        break                   
                    else:
                        sleep(1)
                        continue                
                else:                    
                    continue
```
Below is a sample of what it looks like after the code is run:

<img src='./Images/stoploss.jpg' width="800">

Notice that there are a couple of crucial parameters that can be optimized.

While a blanket solution such as 12 pips difference from current price for the buy and sell stop can be used, we can calculate an optimum combination by leveraging on past data.

## 2. Optimization of strategy parameters



One drawback of this strategy is that it is relatively difficult to backtest, as it would call for data granularity up to the tick level. Unfortunately, I was personally unable to locate any data at the tick level with prices which I could afford in terms of price.

However, even with minute level data, optimization can still be done on two parameters:
1. Currency pair to scalp
2. Difference set from current price for sell stop and buy stop

For the currency pair, by calculating the max change of price within the timeframe, we can find the currency pair that has the highest volatility and highest change in price, which is ideal for the strategy. For the difference set from current price, we can also easily find the ideal number of pips to set such that we can try to  avoid a whipsaw, which would cause us to instantly stop out.

Below is the code used to calculate the parameters:

In [1]:
import pandas as pd
from datetime import datetime

# This is the pool of potential pairs
pairs = ("EURUSD", "USDCAD", "AUDUSD", "GBPUSD", "USDJPY", "USDCHF")

# 2018 NFP dates
dates = ((2018,1,5),(2018,2,2),(2018,3,9),(2018,4,6),(2018,5,4),(2018,6,1),
          (2018,7,6),(2018,8,3),(2018,9,7),(2018,10,5),(2018,11,2),(2018,12,7))

# Empty dict, will contain all of the results
resultdict = {}

In [3]:
# Double nested loop so runs through all the dates for each pair
for j in pairs:
    for i in dates:
        
        # Reading the file
        filename = j + "_out_2018.csv"
        df = pd.read_csv(filename, index_col = 'datetime')
        df.index = pd.to_datetime(df.index)
        
        # Empty lists to be appended later
        highlist = []
        lowlist = []
        
        # Defining ranges, 14:30 is the news strike time for 20:30 news
        startdate = datetime(*i,14,27)
        enddate = datetime(*i,14,36)
        
        # Calculate each minute, then append the list
        for time in pd.date_range(startdate, enddate, freq = "min"):
            # Convert to str for df.loc[] purposes
            timestr = str(time)
            
            # lookup and convert numpy float to native float
            try:
                highno = df.loc[timestr]['High']                
                highfloat = highno.item()
            except:
                highfloat = 0
            
            try:
                lowno = df.loc[timestr]['Low']
                lowfloat = lowno.item()
            except:
                lowfloat = 0
        
            # Append the list
            highlist.append(highfloat)
            lowlist.append(lowfloat)
                
        # From the list find maximum difference (volatility)
        maxdiff = (max(highlist) - min(highlist))*10000
        
        # Updating the final dictionary
        ijkey = str(i) + " " + str(j)        
        resultdict[ijkey] = maxdiff
        
# Save dictionary in dataframe
df2 = pd.DataFrame.from_dict(resultdict, orient='index')
# df2.to_csv("max pip move results.csv", header= False)

print(df2)

                         0
(2018, 1, 5) EURUSD   32.7
(2018, 2, 2) EURUSD   36.5
(2018, 3, 9) EURUSD   27.1
(2018, 4, 6) EURUSD   23.0
(2018, 5, 4) EURUSD   31.7
...                    ...
(2018, 8, 3) USDCHF    8.7
(2018, 9, 7) USDCHF   16.6
(2018, 10, 5) USDCHF  19.3
(2018, 11, 2) USDCHF   2.1
(2018, 12, 7) USDCHF  31.0

[72 rows x 1 columns]


With a bit of further cleaning in excel we get (refer to file in Excel File Output folder for full illustration):

<img src='./Images/excelfileoutput.jpg' width="800">

From this, we can see that in 2018, if we were to scalp NFP, USDCAD would be the ideal choice. To prevent a whipsaw, a value above 15.2 but below 16 would probably be ideal, as it barely gets us out of the range of a whipsaw.

While past performance does not guarantee the future, these calculations can help serve as a benchmark to find a ballpark figure for the values to be set for the buy stop and sell stop, as well as the currency pair to be chosen.